# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PouyaKholdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PouyaKholdi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PouyaKholdi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PouyaKholdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Desaster.db')
df = pd.read_sql_table('CatMess', con=engine)
Y = df.drop(['message', 'original', 'id', 'genre'], axis=1)
X = df['message']

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words('english')]
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\skl

C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidfvect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

In [7]:
Y_pred = pd.DataFrame(data=Y_pred, columns=Y_test.columns)

In [8]:
print(classification_report(Y_test, Y_pred, target_names=Y_test.keys()))

for col in Y_test.columns:
    print('Classification Report for '+ col)
    print(classification_report(Y_test[col], Y_pred[col]))
    print('----------------------------------------------------\n')

C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4994
               request       0.81      0.45      0.58      1090
                 offer       0.00      0.00      0.00        29
           aid_related       0.73      0.61      0.67      2706
          medical_help       0.61      0.11      0.18       509
      medical_products       0.72      0.13      0.22       333
     search_and_rescue       0.66      0.11      0.19       190
              security       0.00      0.00      0.00       117
              military       0.53      0.09      0.16       225
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.36      0.51       414
                  food       0.84      0.48      0.61       756
               shelter       0.84      0.29      0.43       572
              clothing       0.64      0.15      0.25       104
                 money       0.67      

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5715
           1       0.51      0.06      0.10       839

    accuracy                           0.87      6554
   macro avg       0.69      0.53      0.52      6554
weighted avg       0.83      0.87      0.83      6554

----------------------------------------------------

Classification Report for infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6133
           1       0.11      0.00      0.00       421

    accuracy                           0.93      6554
   macro avg       0.52      0.50      0.49      6554
weighted avg       0.88      0.93      0.90      6554

----------------------------------------------------

Classification Report for transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6247
           1       0.69      0.12      0.20 

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidfvect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001BD8512A318>,
                   use_idf=True, vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='a

In [10]:
parameters = [
    {
        'clf': [MultiOutputClassifier(RandomForestClassifier())],
        'clf__estimator__n_estimators': [10, 20]
    },
    {
        'clf': [MultiOutputClassifier(KNeighborsClassifier())],
        'clf__estimator__n_neighbors': [2, 5, 8],
        'clf__estimator__weights': ['uniform', 'distance']
    }
]

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, Y_train)
cv.best_params_

C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'clf': MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='minkowski',
                                                      metric_params=None,
                                                      n_jobs=None, n_neighbors=8,
                                                      p=2, weights='uniform'),
                       n_jobs=None),
 'clf__estimator__n_neighbors': 8,
 'clf__estimator__weights': 'uniform'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
Ycv_pred = cv.predict(X_test)

In [12]:
Ycv_pred = pd.DataFrame(data=Ycv_pred, columns=Y_test.columns)

In [13]:
print(classification_report(Y_test, Ycv_pred, target_names=Y_test.keys()))

for col in Y_test.columns:
    print('Classification Report for '+ col)
    print(classification_report(Y_test[col], Ycv_pred[col]))
    print('----------------------------------------------------\n')

C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\PouyaKholdi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      4994
               request       0.82      0.14      0.24      1090
                 offer       0.00      0.00      0.00        29
           aid_related       0.83      0.11      0.19      2706
          medical_help       0.50      0.00      0.00       509
      medical_products       0.71      0.02      0.03       333
     search_and_rescue       1.00      0.01      0.01       190
              security       0.00      0.00      0.00       117
              military       0.67      0.01      0.02       225
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.05      0.09       414
                  food       0.82      0.06      0.11       756
               shelter       0.90      0.03      0.06       572
              clothing       0.80      0.04      0.07       104
                 money       0.00      

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5715
           1       0.56      0.01      0.01       839

    accuracy                           0.87      6554
   macro avg       0.71      0.50      0.47      6554
weighted avg       0.83      0.87      0.81      6554

----------------------------------------------------

Classification Report for infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6133
           1       0.00      0.00      0.00       421

    accuracy                           0.94      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.88      0.94      0.90      6554

----------------------------------------------------

Classification Report for transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6247
           1       1.00      0.01      0.02 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
#Added a second estimator to the pipeline.
#Removed Stopwords from the messages.

### 9. Export your model as a pickle file

In [15]:
import pickle

CatMessClf = pickle.dump(cv, open('./CatMessClf.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.